In [15]:
import tweepy
import os
from google.cloud import language_v1
import re
import pandas as pd
import plotly.express as px

In [2]:
##API Tweet Credentials
cred = pd.read_csv("credentials.csv")

In [3]:
consumerKey = cred["keys"][0]
consumerSecret = cred["keys"][1]
accessToken = cred["keys"][2]
accessTokenSecret = cred["keys"][3]

In [4]:
#Google cloud Credentials
google_credential = "C:\\Users\\hp\\Desktop\\NLPTwitter\\google_credential.json"

In [ ]:
##Grabing tweets
tweetsPerQry = 1000
maxTweets = 20000
hashtag = "#Bitcoin"

authentication = tweepy.OAuthHandler(consumerKey, consumerSecret)
authentication.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(authentication, wait_on_rate_limit=True)
maxId = -1
tweetCount = 0
listposts = []

while tweetCount < maxTweets:
    if(maxId <= 0):
        newTweets = api.search_tweets(q=hashtag, count=tweetsPerQry, result_type="recent", tweet_mode="extended")
    else:
        newTweets = api.search_tweets(q=hashtag, count=tweetsPerQry, max_id=str(maxId - 1), result_type="recent", tweet_mode="extended")

    if not newTweets:
        print("Tweet Habis")
        break
        
    for tweet in newTweets:
        d={}
        d["text"] = tweet.full_text.encode('utf-8')
        print (d["text"])
        listposts.append(d)
        
    tweetCount += len(newTweets)
    maxId = newTweets[-1].id

In [ ]:
#Cleaning tweets with regex
def cleanTwt(twt):
    twt = re.sub(r'#bitcoin', 'bitcoin', twt)
    twt = re.sub(r'#bitcoin', 'Bitcoin', twt)
    twt = re.sub(r'#[A-Za-z0-9]+', '', twt)
    twt = re.sub(r'\\n', '', twt)
    twt = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&\/\/=]*)', '', twt)
    return twt
listposts=[{"text":cleanTwt(i["text"])} for i in listposts]

In [38]:
##APPLy google nlp to get score and magnitue
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = google_credential
client = language_v1.LanguageServiceClient()

for x in range (len(listposts)):
    try:
        document = language_v1.Document(content=listposts[x]["text"], type_=language_v1.Document.Type.PLAIN_TEXT)
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        sscore = round(sentiment.score,4)
        smag = round(sentiment.magnitude,4)
 
        listposts[x]["score"] = sscore
        listposts[x]["magnitude"] = smag
         
    except Exception as e:
        print(e)
        listposts[x]["score"] = 0
        listposts[x]["magnitude"] = 0

400 The language hi is not supported for document_sentiment analysis.
400 The language ru is not supported for document_sentiment analysis.
400 The language fa is not supported for document_sentiment analysis.
400 The language hi is not supported for document_sentiment analysis.
400 The language fa is not supported for document_sentiment analysis.
400 The language ro is not supported for document_sentiment analysis.
400 The language cs is not supported for document_sentiment analysis.
400 The language ru is not supported for document_sentiment analysis.
400 The language ms is not supported for document_sentiment analysis.
400 The language hi is not supported for document_sentiment analysis.
400 The language fa is not supported for document_sentiment analysis.
400 The language ru is not supported for document_sentiment analysis.
400 The language ro is not supported for document_sentiment analysis.
400 The language el is not supported for document_sentiment analysis.
400 The language el 

In [39]:
#Store it
df = pd.DataFrame(listposts)
df.to_excel("C:\\Users\\hp\\Desktop\\Cv_project\\NLPTwitter\\data\\data.xlsx", index=True)

In [6]:
df = pd.read_excel("C:\\Users\\hp\\Desktop\\NLPTwitter\\data\\data.xlsx", index_col=0)
df = df[["score","magnitude"]]

In [13]:
px.scatter(df, x="score", y="magnitude", color="magnitude", title="Opinion On Bitcoin")

In [12]:
(df["score"].mean(),df["magnitude"].mean())

(0.05196323309021867, 0.3366520131881383)